# URL that is helpful

1. [Preprocessing Technique](https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908)
2. [Convert text into features](https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/)
3. [Using word embedding](https://towardsdatascience.com/word-embeddings-exploration-explanation-and-exploitation-with-code-in-python-5dac99d5d795)

In [1]:
# Download the dataset

# !mkdir -p data/spam_data
# !ls data
# !wget https://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham.tar.bz2
# !wget https://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2
# !wget https://spamassassin.apache.org/old/publiccorpus/20030228_spam.tar.bz2
# !wget https://spamassassin.apache.org/old/publiccorpus/20050311_spam_2.tar.bz2
# !wget https://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2
#
# !tar xvjf 20030228_easy_ham_2.tar.bz2
# !tar xvjf 20030228_easy_ham.tar.bz2
# !tar xvjf 20030228_hard_ham.tar.bz2
# !tar xvjf 20030228_spam.tar.bz2
# !tar xvjf 20050311_spam_2.tar.bz2
#
# !mv easy_ham data/spam_data
# !mv easy_ham_2 data/spam_data
# !mv hard_ham data/spam_data
# !mv spam data/spam_data
# !mv spam_2 data/spam_data

In [2]:
# Download the glove embedding
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip
# !python -m gensim.scripts.glove2word2vec -i glove.6B.300d.txt -o glove.6B.300d.word2vec.txt

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

import os
import glob
import numpy as np
import email

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\59384\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\59384\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Define the file path

path = 'data/spam_data/'
easy_ham_paths = glob.glob(path+'easy_ham/*')
easy_ham_2_paths = glob.glob(path+'easy_ham_2/*')
hard_ham_paths = glob.glob(path+'hard_ham/*')
spam_paths = glob.glob(path+'spam/*')
spam_2_paths = glob.glob(path+'spam_2/*')

In [70]:
def get_email_content(email_path):
    file = open(email_path,encoding='latin1')
    try:
        msg = email.message_from_file(file)
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                return part.get_payload() # prints the raw text
    except Exception as e:
        print(e)

def get_email_content_bulk(email_paths):
    email_contents = [get_email_content(o) for o in email_paths]
    return email_contents

In [6]:
# Split data into train/test
non_spam_path = easy_ham_paths + easy_ham_2_paths + hard_ham_paths
spam_path = spam_paths+spam_2_paths

In [5]:
# Reading the email, just use one email for testing purposes
sample = non_spam_path[0]
# print(sample)
file = open(sample)
try:
    msg = email.message_from_file(file)
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            body = part.get_payload()
            print(body)
except Exception as e:
    print(e)

    Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU at Sun Mar 17 14:55

In [85]:
text = open(non_spam_path[0], 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 5216 characters


In [68]:
non_spam_sentences = []
spam_sentences = []
non_spam_labels = []
spam_labels = []
count = 0

# Extract the body content for non-spam message
for path in non_spam_path:
    file = open(path, encoding="utf8", errors='ignore')
    try:
        msg = email.message_from_file(file)
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True).decode()
                non_spam_sentences.append(body)
                non_spam_labels.append(1)
                count+=1
    except Exception as e:
        print(e)
# Extract the body content for spam message
for path in spam_path:
    file = open(path, encoding="utf8", errors='ignore')
    try:
        msg = email.message_from_file(file)
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload(decode=True).decode()
                spam_sentences.append(body)
                spam_labels.append(0)
                count+=1
    except Exception as e:
        print(e)

# Expect: the number of spam_sentences == number of spam_labels
print(f'Total number of message: {count}')
print(f"number of Non-Spam sentence: {len(non_spam_sentences)}== number of non_spam_labels: {len(non_spam_labels)}")
print(f"number of Spam sentence: {len(spam_sentences)}== number of Spam_labels: {len(spam_labels)}")


'utf-8' codec can't decode byte 0xb4 in position 93: invalid start byte
'utf-8' codec can't decode byte 0xe8 in position 565: invalid continuation byte
'utf-8' codec can't decode byte 0xe9 in position 2073: invalid continuation byte
'utf-8' codec can't decode byte 0xe9 in position 1965: invalid continuation byte
'utf-8' codec can't decode byte 0xe9 in position 1456: invalid continuation byte
'utf-8' codec can't decode byte 0xe9 in position 1457: invalid continuation byte
'utf-8' codec can't decode byte 0xab in position 2130: invalid start byte
'utf-8' codec can't decode byte 0xb7 in position 1139: invalid start byte
'utf-8' codec can't decode byte 0xe9 in position 1652: invalid continuation byte
'utf-8' codec can't decode byte 0xab in position 1056: invalid start byte
'utf-8' codec can't decode byte 0xab in position 367: invalid start byte
'utf-8' codec can't decode byte 0xab in position 454: invalid start byte
'utf-8' codec can't decode byte 0xe9 in position 1452: invalid continuation

In [69]:
# Define the variables
vocab_size = 1000 # Size of word corpus
embedding_dim = 16 # dimension of output vector
max_length = 120 # Maximum length of token
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<ooV>'
training_portion = .8


In [112]:
train_size = int(len(non_spam_labels) * training_portion)
# Split the trainign and testing dataset
x_train = non_spam_sentences[:train_size]
y_train = non_spam_labels[:train_size]
x_test = non_spam_sentences[train_size:]
y_test = non_spam_labels[train_size:]
print("Expected the number of len(non_spam_labels) equals to the sum of len(train_labels)+len(valid_labels)")
print(f'Expected {len(y_train)}+{len(y_test)}=={len(non_spam_labels)}')

train_size = int(len(spam_labels) * training_portion)
x_train += spam_sentences[:train_size]
y_train += spam_labels[:train_size]
x_test += spam_sentences[train_size:]
y_test += spam_labels[train_size:]

print('\n')
print(f'number of x_train: {len(x_train)}')
print(f'number of y_train: {len(y_train)}')
print(f'number of x_test: {len(x_test)}')
print(f'number of y_test: {len(y_test)}')

Expected the number of len(non_spam_labels) equals to the sum of len(train_labels)+len(valid_labels)
Expected 3228+807==4035


number of x_train: 4004
number of y_train: 4004
number of x_test: 1001
number of y_test: 1001


In [113]:
print(f'Number of non-spam file: {len(non_spam_path)}')
print(f'Number of spam file: {len(spam_path)}')

# Expected:
# Number of non-spam file: 4153
# Number of spam file: 1898

Number of non-spam file: 4153
Number of spam file: 1898


In [114]:
# Inspect the message content
print(f"Non-Spam sample: \n{non_spam_sentences[0]}")
# print(f"Spam sample: {spam_sentences[0]}")

Non-Spam sample: 
    Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchsia.cs.mu.OZ.AU a

In [115]:
## Process word by word
# def remove_null(datas,labels):
#     not_null_idx = [i for i,o in enumerate(datas) if o is not None]
#     return np.array(datas)[not_null_idx],np.array(labels)[not_null_idx]
# x_train,y_train = remove_null(x_train,y_train)
# x_test,y_test = remove_null(x_test,y_test)

print('\n')
print(f'number of x_train: {len(x_train)}')
print(f'number of y_train: {len(y_train)}')
print(f'number of x_test: {len(x_test)}')
print(f'number of y_test: {len(y_test)}')



number of x_train: 4004
number of y_train: 4004
number of x_test: 1001
number of y_test: 1001


In [127]:
# Remove stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [128]:
#----------------------Testing the cleaning operation on single sample--------------------
msg = x_train[0]
# The upper and lower case should have no impact to the result(neg/pos), so we should convert all chars to lower case, to reduce the overhead of traning process.
msg.lower()
# Using regex to remove any not-chars or \', hyperlinks, emails addr, and same header information
pattern = '[^a-zA-Z\']|\_+|http\S+|\S*@\S*\s?|Date:.*|From:.*|Message.*'
replace = " "
msg = re.sub(pattern, replace, msg).strip()
# Calling re.sub again to remove those redundant whitespace
msg = re.sub('[\r\n\t ]+', replace, msg).strip()
msg
# msg = [i for i in msg if i not in stopwords]


"I can't reproduce this error For me it is very repeatable like every time without fail This is the debug log of the pick happening Pick It exec pick inbox list lbrace lbrace subject ftp rbrace rbrace sequence mercury exec pick inbox list lbrace lbrace subject ftp rbrace rbrace sequence mercury Ftoc PickMsgs hit Marking hits tkerror syntax error in expression int Note if I run the pick command by hand delta pick inbox list lbrace lbrace subject ftp rbrace rbrace sequence mercury hit That's where the hit comes from obviously The version of nmh I'm using is delta pick version pick nmh compiled on fuchsia cs mu OZ AU at Sun Mar ICT And the relevant part of my mh profile delta mhparam pick seq sel list Since the pick command works the sequence actually both of them the one that's explicit on the command line from the search popup and the one that comes from mh profile do get created kre ps this is still using the version of the code form a day ago I haven't been able to reach the cvs repos

In [129]:
#----------------------Applying the cleaning operation on all trainign data --------------------
def msg_cleaning(msg):
    # The upper and lower case should have no impact to the result(neg/pos), so we should convert all chars to lower case, to reduce the overhead of traning process.
    msg.lower()
    # Using regex to remove any not-chars or \', hyperlinks, emails addr, and same header information
    pattern = '[^a-zA-Z\']|\_+|http\S+|\S*@\S*\s?|Date:.*|From:.*|Message.*'
    replace = " "
    msg = re.sub(pattern, replace, msg).strip()
    # Calling re.sub again to remove those redundant whitespace
    msg = re.sub('[\r\n\t ]+', replace, msg).strip()
    # Remove the any word in stopwords from msg (Those words has no semantic meaning to our model, so we should remove them to reduce the training overhead)
    msg = msg.split(' ')
    msg = [word for word in msg if word not in stopwords]
    msg = " ".join(msg)
    return msg

In [ ]:
msg = x_train[0]
msg = msg_cleaning(msg)
msg = msg.split(' ')
msg = [word for word in msg if word not in stopwords]
msg = " ".join(msg)
msg

In [ ]:
# Process the data in the word-by-word fasion: Spcifically, for each word, if it's in the set of stopwords, remove it.
count = 0
for idx, msg in enumerate(x_train):
    x_train[idx] = msg_cleaning(msg)
    count+=1
    print(count)
for idx, msg in enumerate(x_test):
    x_test[idx] = msg_cleaning(msg)
    count+=1
    print(count)
print('\n')
print(f'number of x_train: {len(x_train)}')
print(f'number of y_train: {len(y_train)}')
print(f'number of x_test: {len(x_test)}')
print(f'number of y_test: {len(y_test)}')
# Now the msg looks much more clean, let's begin the Tokenization process.

In [ ]:
lemmatizer = WordNetLemmatizer()
msg = [lemmatizer.lemmatize(o) for o in msg]

In [ ]:
# Inspect the message content
print(f"Non-Spam sample: \n{non_spam_sentences[0]}")

In [34]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

c:\users\59384\anaconda3\envs\facecourse-py3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [35]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [36]:
x_train = [word_tokenize(o) for o in train_sentences]
x_test = [word_tokenize(o) for o in valid_sentences]

In [37]:
def remove_stop_words(words):
    result = [i for i in words if i not in ENGLISH_STOP_WORDS]
    return result

In [38]:
def word_stemmer(words):
    return [stemmer.stem(o) for o in words]

In [39]:
def word_lemmatizer(words):
    return [lemmatizer.lemmatize(o) for o in words]

In [ ]:
def clean_token_pipeline(words):
    cleaning_utils = [remove_stop_words,word_stemmer,word_lemmatizer]
    for o in cleaning_utils:
        words = o(words)
    return words

In [41]:
x_train = [clean_token_pipeline(o) for o in x_train]
x_test = [clean_token_pipeline(o) for o in x_test]

In [42]:
# Play around with Word2vector

In [43]:
from gensim.models import KeyedVectors

w2v = KeyedVectors.load_word2vec_format('glove.6B.300d.word2vec.txt',binary=False)

In [44]:
w2v.most_similar('king')

[('queen', 0.6336468458175659),
 ('prince', 0.619662344455719),
 ('monarch', 0.5899620652198792),
 ('kingdom', 0.5791266560554504),
 ('throne', 0.5606487989425659),
 ('ii', 0.5562329292297363),
 ('iii', 0.5503199100494385),
 ('crown', 0.5224862694740295),
 ('reign', 0.521735429763794),
 ('kings', 0.5066401362419128)]

In [45]:
# King + Woman - Man = ?
w2v.most_similar(['king','woman'],negative=['man'],topn=1)

[('queen', 0.6713277101516724)]

In [46]:
w2v.doesnt_match("england china vietnam laos".split())

c:\users\59384\anaconda3\envs\facecourse-py3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'england'

In [47]:
w2v.doesnt_match("fish shark cat whale".split())

'cat'

In [48]:
# Extract Features from Words

## Use Glove Pretrain

In [49]:
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In c:\users\59384\anaconda3\envs\facecourse-py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\users\59384\anaconda3\envs\facecourse-py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\users\59384\anaconda3\envs\facecourse-py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In c:\users\59384\anaconda3\envs\facecourse-py3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\users\

In [50]:
maxlen = 2000
max_features = 50000

In [51]:
EMBEDDING_FILE = 'glove.6B.300d.txt'
tokenizer = Tokenizer(num_words=max_features)

In [52]:
tokenizer.fit_on_texts(x_train)

In [53]:
x_train_features = np.array(tokenizer.texts_to_sequences(x_train))
x_test_features = np.array(tokenizer.texts_to_sequences(x_test))

x_train_features = pad_sequences(x_train_features,maxlen=maxlen)
x_test_features = pad_sequences(x_test_features,maxlen=maxlen)

In [54]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 6148: character maps to <undefined>

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.layers[1].trainable = False

In [ ]:
history = model.fit(x_train_features,y_train, batch_size=512, epochs=20, 
          validation_data=(x_test_features, y_test))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# Neural Network w Embedding From Scratch

In [ ]:
# Neural Network w Embedding From Scratch

In [ ]:
## some config values 
embed_size = 100 # how big is each word vector
max_feature = 50000 # how many unique words to use (i.e num rows in embedding vector)
max_len = 2000 # max number of words in a question to use

In [ ]:
tokenizer = Tokenizer(num_words=max_feature)

In [ ]:
tokenizer.fit_on_texts(x_train)

In [ ]:
x_train_features = np.array(tokenizer.texts_to_sequences(x_train))
x_test_features = np.array(tokenizer.texts_to_sequences(x_test))

x_train_features = pad_sequences(x_train_features,maxlen=max_len)
x_test_features = pad_sequences(x_test_features,maxlen=max_len)

In [ ]:
embed_size = 100

inp = Input(shape=(max_len,))
x = Embedding(max_feature, embed_size)(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

history = model.fit(x_train_features, y_train, batch_size=512, epochs=20, validation_data=(x_test_features, y_test))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
## Neural Network